In [1]:
import trimesh
import logging
logger = logging.getLogger("trimesh")
logger.setLevel(logging.ERROR)

import os
from os import path
import h5py

from src.data_gen_utils import *

import time
import sys
sys.path.append('../../../rai-fork/rai/ry')

import libry as ry
import multiprocessing as mp

In [4]:
def inCollision(C, frames1, frames2):
    for f1 in frames1:
        for f2 in frames2:
            y = -C.evalFeature(ry.FS.pairCollision_negScalar, [f1, f2])[0]
            if y < 0:
                return True
    return False

def signed_distance(C, frames1, frames2):
    dist = np.inf
    for f1 in frames1:
        for f2 in frames2:
            y = -C.evalFeature(ry.FS.pairCollision_negScalar, [f1, f2])[0]
            dist = min(dist, y)
    return dist

def isFeasible(C):
    C2 = ry.Config()
    C2.copy(C)
    S = C2.simulation(ry.SimulatorEngine.bullet, 0)
    S.setGravity(np.zeros(3))
    S.closeGripper("gripper", objFrameName="mug")
    tau = 0.01
    for _ in range(200):
        S.step([], tau, ry.ControlMode.none)
    f = S.getGripperIsGrasping("gripper")
    S = 0
    return f

def get_feasible_grasp(N, mesh_filename, mesh_coll_filename, mass, com, view=False):
    feasible_poses = []
    object_trimesh = trimesh.load(mesh_filename)
    tau = 0.01
    
    C = ry.Config()
    C.addFile('gripperWorld.g')
    gripperFrames = ['gripper',
                     'L_finger', 'L_finger_1', 'L_finger_2', 'L_finger_3',
                     'R_finger', 'R_finger_1', 'R_finger_2', 'R_finger_3']
    mug = C.addMeshFrame(mesh_coll_filename, 'mug', mass=mass, com=com).setPosition(np.array([0,0,0.8])+com)
    mugFrames = []
    print(C.getFrameNames())
    for fname in C.getFrameNames():
        if fname[:3] == 'mug':
            C.frame(fname).setContact(1)
            mugFrames.append(fname)

    gripper = C.frame('gripper').setPosition([0,0,0.8])
    gripperCenter = C.frame('gripperCenter')
    if view: 
        V = ry.ConfigurationViewer()
        p1f = C.addFrame('p1').setShape(ry.ST.sphere, [0.01]).setColor([1,0,0,0.8])
        p2f = C.addFrame('p2').setShape(ry.ST.sphere, [0.01]).setColor([0,0,1,0.8])

    m_gripperCenter = C.addFrame('m_gripperCenter').setShape(ry.ST.marker, [0.])
    m_gripper = C.addFrame('m_gripper', 'm_gripperCenter').setShape(ry.ST.marker, [0.])
    # m_gripper.setRelativeAffineMatrix(np.linalg.inv(gripperCenter.getRelativeAffineMatrix()))
    m_gripper.setAffineMatrix(np.linalg.inv(gripperCenter.getRelativeAffineMatrix())@m_gripperCenter.getAffineMatrix())

    S = C.simulation(ry.SimulatorEngine.bullet, 0)
    fInit = C.getFrameState()
    
    while True:
        # p1s, inds_p1 = object_trimesh.sample(10000, True)
        p1s, inds_p1 = trimesh.sample.sample_surface_even(object_trimesh, N)

        for p1, idx_p1 in zip(p1s, inds_p1):
            normal_p1 = object_trimesh.face_normals[idx_p1]
            while True:
                ray_direction = -normal_p1+0.1*np.random.randn(3)
                ray_direction /= np.linalg.norm(ray_direction)
                if abs(angle(normal_p1, -ray_direction)) < 30.:
                    break
            p2s, _, inds_p2 = object_trimesh.ray.intersects_location(ray_origins=p1.reshape(1,3),
                                                                     ray_directions=ray_direction.reshape(1,3))

            for p2, idx_p2 in zip(p2s, inds_p2):
                normal_p2 = object_trimesh.face_normals[idx_p2]
                if abs(angle(normal_p2, ray_direction)) > 30. or np.linalg.norm(p1-p2)>0.07:
                    continue

                pos = (p1 + p2)/2
                vectorX = ray_direction
                for _ in range(10):
                    vectorY = np.random.randn(3)
                    vectorY -= np.dot(vectorY, vectorX)*vectorX
                    vectorY /= np.linalg.norm(vectorY)
                    rot = np.stack([vectorX, vectorY, np.cross(vectorX, vectorY)], axis=1)
                    T = np.eye(4)
                    T[:3,:3] = rot
                    T[:3,3] = pos + np.array([0,0,.8])

                    m_gripperCenter.setAffineMatrix(T)
                    gripper.setAffineMatrix(m_gripper.getAffineMatrix())
#                     V.setConfiguration(C)
                    if not inCollision(C, gripperFrames, mugFrames) and isFeasible(C):
                        print('good')
                        pose = np.hstack([gripperCenter.getPosition()-np.array([0,0,.8]),
                                          gripperCenter.getQuaternion()])
                        feasible_poses.append(pose)
                        if view:
                            p1f.setPosition(p1+ np.array([0,0,.8]))
                            p2f.setPosition(p2+ np.array([0,0,.8]))
                            V.setConfiguration(C)
                        if len(feasible_poses) == N:
                            return np.array(feasible_poses)
                    C.setFrameState(fInit)

In [5]:
load_dir = 'data/object'
mesh_dir = 'data/meshes'
save_dir = 'data/grasp'

def start_process():
    print('Starting', mp.current_process().name)

    
def saveGraspData(filename):
    print(filename)
    
    data_obj = h5py.File(path.join(load_dir, filename), mode='r')
    name = data_obj['filename'][()].decode()
    size = data_obj['size'][()]
    mass = data_obj['mass'][()]
    com = data_obj['com'][:]
    data_obj.close()
    
    mesh_filename = path.join(mesh_dir, name)  
    mesh_coll_filename = path.join(mesh_dir+'_coll', name)   
    
    feasible_poses = get_feasible_grasp(1000, 
                                        mesh_filename, 
                                        mesh_coll_filename, 
                                        mass, 
                                        com, 
                                        view=False)
    data = h5py.File(path.join(save_dir, filename), mode='w')
    data.create_dataset("pose", data=feasible_poses, dtype=np.float32)
    data.close()
    
    
filename_list = [fn for fn in os.listdir(load_dir) if fn.endswith('.hdf5')]
# saveGraspData(filename_list[0])
print(len(filename_list))
pool = mp.Pool(processes=16,
               initializer=start_process)
outputs = pool.map(saveGraspData, filename_list)

pool.close() # no more tasks
pool.join()  # wrap up current tasks

547b3284875a6ce535deb3b0c692a2a.hdf5
['world', 'gripper', 'L_finger', 'gripper>L_finger', 'L_finger_coll1', 'L_finger_coll2', 'L_finger_1', 'L_finger_2', 'L_finger_3', 'R_finger', 'gripper>R_finger', 'R_finger_coll1', 'R_finger_coll2', 'R_finger_1', 'R_finger_2', 'R_finger_3', 'gripper_coll', 'gripperCenter', '|gripper', 'mug', 'mug_1', 'mug_2', 'mug_3', 'mug_4', 'mug_5', 'mug_6', 'mug_7', 'mug_8', 'mug_9', 'mug_10', 'mug_11', 'mug_12', 'mug_13', 'mug_14', 'mug_15', 'mug_16', 'mug_17', 'mug_18', 'mug_19', 'mug_20', 'mug_21', 'mug_22', 'mug_23', 'mug_24', 'mug_25', 'mug_26', 'mug_27', 'mug_28', 'mug_29', 'mug_30', 'mug_31', 'mug_32', 'mug_33', 'mug_34', 'mug_35', 'mug_36', 'mug_37', 'mug_38', 'mug_39', 'mug_40', 'mug_41', 'mug_42', 'mug_43', 'mug_44', 'mug_45']
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good

KeyboardInterrupt: 

In [15]:
def inCollision(gripperFrames, mugFrames, C):
    for g in gripperFrames:
        for m in mugFrames:
            y = C.evalFeature(ry.FS.pairCollision_negScalar, [g,m])[0]
            if y > 0:
                return True
    return False


def saveGraspData(filename):
    print(filename)
    
    data_obj = h5py.File(path.join(load_dir, filename), mode='r')
    name = data_obj['filename'][()]
    size = data_obj['size'][()]
    mass = data_obj['mass'][()]
    
    com = data_obj['com'][:]
    data_obj.close()
    
    mesh_filename = path.join(mesh_dir, name)  
    mesh_coll_filename = path.join(mesh_dir+'_coll', name)   
    
    feasible_poses = get_feasible_grasp(1000, 
                                        mesh_filename, 
                                        mesh_coll_filename, 
                                        mass, 
                                        com, 
                                        view=False)
    data = h5py.File(path.join(save_dir, filename), mode='w')
    data.create_dataset("pose", data=feasible_poses, dtype=np.float32)
    data.close()
    
    
filename_list = [fn for fn in os.listdir(load_dir) if fn.endswith('.hdf5')]
grasp_list = [fn for fn in os.listdir(save_dir) if fn.endswith('.hdf5')]
for filename in filename_list:
    if filename in grasp_list:
        continue
    if filename == "b88bcf33f25c6cb15b4f129f868dedb.hdf5":
        continue
    saveGraspData(filename)

c60f62684618cb52a4136492f17b9a59.hdf5


CompletedProcess(args=['mv', 'meshes_coll/b88bcf33f25c6cb15b4f129f868dedb.obj', 'bad/meshes_coll'], returncode=1)

In [3]:
data_dir = 'mug_data'
filename_list = [fn for fn in os.listdir(data_dir) if fn.endswith('.hdf5')]
for i, filename in enumerate(filename_list[0:]):
    print('{}/{}: {}'.format(i, len(filename_list), filename))
    
    data = h5py.File('mug_data/'+filename, mode='a')
    
    name = data['object/filename'][()].decode()
    size = data['object/size'][()]
    mass = data['object/mass'][()]
    com = data['object/com'][:]
    
    mesh_filename = path.join(data_dir, 'meshes', name)  
    mesh_coll_filename = path.join(data_dir, 'meshes_coll', name)    

    
    feasible_poses = get_feasible_grasp(1000, 
                                        mesh_filename, 
                                        mesh_coll_filename, 
                                        mass, 
                                        com, 
                                        view=False)
    
    grasp = data.create_group("grasp")
    grasp.create_dataset("pose", data=feasible_poses, dtype=np.float32)
    
    data.close()


0/137: 547b3284875a6ce535deb3b0c692a2a.hdf5


KeyboardInterrupt: 

2d10421716b16580e45ef4135c266a12.hdf55c7c4cb503a757147dbda56eabff0c47.hdf5

547b3284875a6ce535deb3b0c692a2a.hdf5
5c48d471200d2bf16e8a121e6886e18d.hdf5
6aec84952a5ffcf33f60d03e1cb068dc.hdf5
5fe74baba21bba7ca4eec1b19b3a18f8.hdf5
e9499e4a9f632725d6e865157050a80e.hdf5
b88bcf33f25c6cb15b4f129f868dedb.hdf5
8570d9a8d24cb0acbebd3c0c0c70fb03.hdf5
e79d807e1093c6174e716404e6ec3a5f.hdf5
6faf1f04bde838e477f883dde7397db2.hdf5
6e884701bfddd1f71e1138649f4c219.hdf5
dfa8a3a0c8a552b62bc8a44b22fcb3b9.hdf5
fad118b32085f3f2c2c72e575af174cd.hdf5
73b8b6456221f4ea20d3c05c08e26f.hdf5
79e673336e836d1333becb3a9550cbb1.hdf5
bed29baf625ce9145b68309557f3a78c.hdf5
9961ccbafd59cb03fe36eb2ab5fa00e0.hdf5
40f9a6cc6b2c3b3a78060a3a3a55e18f.hdf5
1305b9266d38eb4d9f818dd0aa1a251.hdf5
b18bf84bcd457ffbc2868ebdda32b945.hdf5
141f1db25095b16dcfb3760e4293e310.hdf5
9c930a8a3411f069e7f67f334aa9295c.hdf5
1d18255a04d22794e521eeb8bb14c5b3.hdf5
6500ccc65e210b14d829190312080ea3.hdf5
c6bc2c9770a59b5ddd195661813efe58.hdf5
62634df2ad8f19b87d

In [10]:
mp.cpu_count()

8

In [4]:
object_trimesh = trimesh.load(mesh_filename)
C = ry.Config()
C.addFile('gripperWorld.g')
gripperFrames = ['gripper',
                 'L_finger', 'L_finger_1', 'L_finger_2', 'L_finger_3',
                 'R_finger', 'R_finger_1', 'R_finger_2', 'R_finger_3']
mug = C.addMeshFrame(mesh_coll_filename, 'mug', mass=mass, com=com).setPosition([0,0,0.8])
mugFrames = []
for fname in C.getFrameNames():
    if fname[:3] == 'mug':
        C.frame(fname).setContact(1)
        mugFrames.append(fname)
        
gripper = C.frame('gripper').setPosition([0,0,0.8])
gripperCenter = C.frame('gripperCenter')
V = ry.ConfigurationViewer()
p1f = C.addFrame('p1').setShape(ry.ST.sphere, [0.01]).setColor([1,0,0,0.8])
p2f = C.addFrame('p2').setShape(ry.ST.sphere, [0.01]).setColor([0,0,1,0.8])

m_gripperCenter = C.addFrame('m_gripperCenter').setShape(ry.ST.marker, [0.])
m_gripper = C.addFrame('m_gripper', 'm_gripperCenter').setShape(ry.ST.marker, [0.])
# m_gripper.setRelativeAffineMatrix(np.linalg.inv(gripperCenter.getRelativeAffineMatrix()))
m_gripper.setAffineMatrix(np.linalg.inv(gripperCenter.getRelativeAffineMatrix())@m_gripperCenter.getAffineMatrix())

# p1s, inds_p1 = object_trimesh.sample(10000, True)
p1s, inds_p1 = trimesh.sample.sample_surface_even(object_trimesh, 10000)

for p1, idx_p1 in zip(p1s, inds_p1):
    normal_p1 = object_trimesh.face_normals[idx_p1]
    while True:
        ray_direction = -normal_p1+0.1*np.random.randn(3)
        ray_direction /= np.linalg.norm(ray_direction)
        if abs(angle(normal_p1, -ray_direction)) < 3.:
            break
    p2s, _, inds_p2 = object_trimesh.ray.intersects_location(ray_origins=p1.reshape(1,3),
                                                             ray_directions=ray_direction.reshape(1,3))
        
        
        
    for p2, idx_p2 in zip(p2s, inds_p2):
        normal_p2 = object_trimesh.face_normals[idx_p2]
        if abs(angle(normal_p2, ray_direction)) > 3. or np.linalg.norm(p1-p2)>0.1:
            continue
        
        pos = (p1 + p2)/2
        vectorX = ray_direction
        
        for _ in range(10):
            vectorY = np.random.randn(3)
            vectorY -= np.dot(vectorY, vectorX)*vectorX
            vectorY /= np.linalg.norm(vectorY)
            rot = np.stack([vectorX, vectorY, np.cross(vectorX, vectorY)], axis=1)
            T = np.eye(4)
            T[:3,:3] = rot
            T[:3,3] = pos + np.array([0,0,.8])

            m_gripperCenter.setAffineMatrix(T)
            gripper.setAffineMatrix(m_gripper.getAffineMatrix())

            p1f.setPosition(p1+ np.array([0,0,.8]))
            p2f.setPosition(p2+ np.array([0,0,.8]))
            
            if not inCollision(gripperFrames, mugFrames, C):
                V.setConfiguration(C)
                
                C2 = ry.Config()
                C2.copy(C)
                S = C2.simulation(ry.SimulatorEngine.bullet, 0)
                S.setGravity(np.zeros(3))
                S.closeGripper("gripper", thetaFrictionCone = 10., objFrameName="mug")
                tau = 0.01
                for _ in range(200):
                    S.step([], tau, ry.ControlMode.none)
                    
                if S.getGripperIsGrasping("gripper"):
                    input()
            
                


KeyboardInterrupt: Interrupted by user